# Pandas DataFrames (`pd.DataFrame`)

The pandas dataframe is the workhorse of most data science and analytics projects.  The dataframe represents the data you're working with as a table.  However, the flexibility of the dataframe is that each row **and** column is represented as a pandas Series, which allows for many powerful ways to mess around with the data

In [1]:
import pandas as pd
import numpy as np
import requests

First let's get some data so we can see what we can do with a data frame.  Don't worry about exactly what this function is doing, we will go over it in a bit.

In [2]:
def get_data(token):
    res = requests.get(
        f'https://api.cryptowat.ch/markets/coinbase-pro/{token}usd/ohlc',
        params={
            'periods': '3600',
            'after': str(int(pd.Timestamp('2021-12-01').timestamp()))
        }
    )

    df = pd.DataFrame(
        res.json()['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
    df['ts'] = pd.to_datetime(df.ts, unit='s')
    df['token'] = token
    
    return df


In [3]:
tokens = ['BTC', 'ETH', 'SOL', 'AAVE', 'COMP']

Don't worry too much about what is going on in the function below - we'll briefly go over it as it showcases the power of python, but it's not necessary for the class

In [4]:
dfs = [
    (lambda x: x.assign(chain=np.where(x.token.isin(['ETH', 'AAVE', 'COMP']), np.full(x.shape[0], 'ETH'), x.token)))(get_data(token)) 
    for token in tokens
]

In [5]:
df_base = pd.concat(get_data(token) for token in tokens)
df_base['chain'] = np.where(df_base.token.isin(['ETH', 'AAVE', 'COMP']), np.full(df_base.shape[0], 'ETH'), df_base.token)


In [6]:
df = df_base.set_index('ts')

## Understanding the data frame

After loading the data in our data frame, we can now inspect what's inside.  We'll need to do this as often the data we will store will be impossible to inspect row by row, and we will need to check that our data loading was correct

Let's check some basic properties of the data set:

We can see how many rows and columns this data frame has, and total number of data points

In [7]:
df.shape

(1460, 8)

In [8]:
df.size

11680

We can see what the first 5 rows looks like:

In [9]:
df.head()

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,BTC
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,BTC
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,BTC
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,BTC
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,BTC


...and the last 5 rows

In [10]:
df.tail()

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-12 23:00:00,199.71,200.00,197.97,199.27,618.775,123024.49025,COMP,ETH
2021-12-13 00:00:00,199.27,200.05,198.30,198.62,296.920,59113.82992,COMP,ETH
2021-12-13 01:00:00,198.52,200.81,195.49,196.24,3363.459,663783.90199,COMP,ETH
2021-12-13 02:00:00,196.31,196.32,191.80,191.86,1901.019,369748.01226,COMP,ETH
2021-12-13 03:00:00,191.69,193.47,190.95,192.11,454.858,87473.43382,COMP,ETH


We can also see a general overview of the schema (column name, data and data type) of the data

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1460 entries, 2021-12-01 00:00:00 to 2021-12-13 03:00:00
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       1460 non-null   float64
 1   high       1460 non-null   float64
 2   low        1460 non-null   float64
 3   close      1460 non-null   float64
 4   volume     1460 non-null   float64
 5   volumeUSD  1460 non-null   float64
 6   token      1460 non-null   object 
 7   chain      1460 non-null   object 
dtypes: float64(6), object(2)
memory usage: 102.7+ KB


as well as descriptive statistics about every column

In [12]:
df.describe()

,open,high,low,close,volume,volumeUSD
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1.460000e+03
mean,11209.356016,11276.938413,11131.135530,11202.778737,16987.659321,1.916371e+07
std,20101.662023,20218.374716,19966.832587,20089.482510,34912.560290,2.837019e+07
min,165.280000,168.820000,161.510000,165.320000,67.178000,1.821233e+04
25%,193.647500,195.716500,191.797500,193.491750,784.116811,3.403129e+05
50%,238.113000,239.794500,235.296000,237.044500,2206.631000,1.088089e+07
75%,4413.157500,4432.662500,4377.775000,4411.615000,12528.347007,2.850002e+07
max,58664.400000,59118.840000,58445.530000,58664.400000,534212.095000,3.988035e+08


## DataFrame Indexing

Indexing in data frames works very similar to Series, however there are now two "axes" that we can operate on - rows and columns.  For example, using `[*]` for indexing (like in series) by default will operate on columns:

In [13]:
df['open']

ts
2021-12-01 00:00:00    57321.41
2021-12-01 01:00:00    56998.35
2021-12-01 02:00:00    57618.55
2021-12-01 03:00:00    57029.79
2021-12-01 04:00:00    57306.55
                         ...   
2021-12-12 23:00:00      199.71
2021-12-13 00:00:00      199.27
2021-12-13 01:00:00      198.52
2021-12-13 02:00:00      196.31
2021-12-13 03:00:00      191.69
Name: open, Length: 1460, dtype: float64

however using `.loc[*]` will allow you to access rows:

In [14]:
df.loc['2021-12-01 01:00:00']

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 01:00:00,56998.350,57726.450,56705.060,57616.410,599.791578,3.437153e+07,BTC,BTC
2021-12-01 01:00:00,4636.430,4736.900,4605.490,4729.100,13819.061610,6.487693e+07,ETH,ETH
2021-12-01 01:00:00,208.716,211.773,207.821,211.507,92606.555000,1.946153e+07,SOL,SOL
2021-12-01 01:00:00,257.149,266.249,255.270,264.816,5752.541000,1.516148e+06,AAVE,ETH
2021-12-01 01:00:00,278.650,283.800,276.360,283.440,817.668000,2.292746e+05,COMP,ETH


and `.iloc[*]` will get you positional rows

In [15]:
df.iloc[0]

open               57321.41
high               57451.05
low                56814.34
close              56987.97
volume           388.482022
volumeUSD    22184300.66241
token                   BTC
chain                   BTC
Name: 2021-12-01 00:00:00, dtype: object

we can also get to the last row easily

In [16]:
df.iloc[-1]

open              191.69
high              193.47
low               190.95
close             192.11
volume           454.858
volumeUSD    87473.43382
token               COMP
chain                ETH
Name: 2021-12-13 03:00:00, dtype: object

or return it as a data frame instead of a Series

In [17]:
df.iloc[[-1]]

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-13 03:00:00,191.69,193.47,190.95,192.11,454.858,87473.43382,COMP,ETH


**note**: `df.loc[0]` will not work, as this is accessing via index

---
**note**: Also, the index operators will return a `pd.Series` if there's 1 row returned, or a new `pd.DataFrame` if multiple rows are returned, e.g.:

In [18]:
type(df.iloc[0])

pandas.core.series.Series

In [19]:
type(df.loc['2021-12-01 01:00:00']) # 5 rows returned

pandas.core.frame.DataFrame

we can convert a Series to a DataFrame anytime by using the `.to_frame()` method on the Series object.  This will turn the Series to a DataFrame, using the `Series.name` as the column name

In [20]:
df.iloc[0].to_frame()

,2021-12-01
open,57321.41
high,57451.05
low,56814.34
close,56987.97
volume,388.482022
volumeUSD,22184300.66241
token,BTC
chain,BTC


---

In addition, we can select on multiple columns and rows in various ways:

In [21]:
df[['open', 'close']]

,open,close
ts,,
2021-12-01 00:00:00,57321.41,56987.97
2021-12-01 01:00:00,56998.35,57616.41
2021-12-01 02:00:00,57618.55,57030.83
2021-12-01 03:00:00,57029.79,57307.59
2021-12-01 04:00:00,57306.55,57404.01
...,...,...
2021-12-12 23:00:00,199.71,199.27
2021-12-13 00:00:00,199.27,198.62
2021-12-13 01:00:00,198.52,196.24


In [22]:
df[0:2]

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,BTC
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,BTC


In [23]:
df.loc['2021-12-01 00:00:00':'2021-12-01 02:00:00']

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 00:00:00,57321.410,57451.050,56814.340,56987.970,388.482022,2.218430e+07,BTC,BTC
2021-12-01 01:00:00,56998.350,57726.450,56705.060,57616.410,599.791578,3.437153e+07,BTC,BTC
2021-12-01 02:00:00,57618.550,57620.000,56972.970,57030.830,591.687200,3.387067e+07,BTC,BTC
2021-12-01 00:00:00,4656.620,4672.430,4624.160,4634.950,6013.006735,2.793321e+07,ETH,ETH
2021-12-01 01:00:00,4636.430,4736.900,4605.490,4729.100,13819.061610,6.487693e+07,ETH,ETH
2021-12-01 02:00:00,4729.100,4729.100,4684.490,4695.780,7491.465440,3.524161e+07,ETH,ETH
2021-12-01 00:00:00,210.312,210.590,208.432,208.676,70031.618000,1.465851e+07,SOL,SOL
2021-12-01 01:00:00,208.716,211.773,207.821,211.507,92606.555000,1.946153e+07,SOL,SOL
2021-12-01 02:00:00,211.506,212.235,210.003,210.868,49728.032000,1.049756e+07,SOL,SOL


In [24]:
df.iloc[0:4]

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,BTC
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,BTC
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,BTC
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,BTC


In [25]:
df.iloc[[0, 4, 10, 50]]

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,BTC
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,BTC
2021-12-01 10:00:00,56945.94,57220.39,56756.48,57131.16,586.052971,3.340768e+07,BTC,BTC
2021-12-03 02:00:00,56569.30,56762.82,56425.99,56545.98,238.584708,1.349612e+07,BTC,BTC


And finally, we can index on both rows and columns at the same time with `.loc`:

In [26]:
df.loc['2021-12-01 00:00:00':'2021-12-01 02:00:00', ['close', 'volume', 'token']]

,close,volume,token
ts,,,
2021-12-01 00:00:00,56987.970,388.482022,BTC
2021-12-01 01:00:00,57616.410,599.791578,BTC
2021-12-01 02:00:00,57030.830,591.687200,BTC
2021-12-01 00:00:00,4634.950,6013.006735,ETH
2021-12-01 01:00:00,4729.100,13819.061610,ETH
2021-12-01 02:00:00,4695.780,7491.465440,ETH
2021-12-01 00:00:00,208.676,70031.618000,SOL
2021-12-01 01:00:00,211.507,92606.555000,SOL
2021-12-01 02:00:00,210.868,49728.032000,SOL


**note**: Given that by default dataframe indices are sequential integers by default, it's good practice to use `.loc` and `.iloc` to index into the data frame to be very clear, for example, let's shuffle our data frame then select:

In [27]:
df_shuffled = df_base.sample(frac=1)

In [28]:
df_shuffled.loc[[0, 2, 3]]

,ts,open,high,low,close,volume,volumeUSD,token,chain
0,2021-12-01 00:00:00,57321.410,57451.050,56814.340,56987.970,388.482022,2.218430e+07,BTC,BTC
0,2021-12-01 00:00:00,257.102,260.775,255.345,257.078,2730.299000,7.039183e+05,AAVE,ETH
0,2021-12-01 00:00:00,210.312,210.590,208.432,208.676,70031.618000,1.465851e+07,SOL,SOL
0,2021-12-01 00:00:00,280.590,281.400,278.300,278.700,207.849000,5.822157e+04,COMP,ETH
0,2021-12-01 00:00:00,4656.620,4672.430,4624.160,4634.950,6013.006735,2.793321e+07,ETH,ETH
2,2021-12-01 02:00:00,264.755,266.187,262.597,263.125,1559.330000,4.124449e+05,AAVE,ETH
2,2021-12-01 02:00:00,211.506,212.235,210.003,210.868,49728.032000,1.049756e+07,SOL,SOL
2,2021-12-01 02:00:00,57618.550,57620.000,56972.970,57030.830,591.687200,3.387067e+07,BTC,BTC
2,2021-12-01 02:00:00,4729.100,4729.100,4684.490,4695.780,7491.465440,3.524161e+07,ETH,ETH
2,2021-12-01 02:00:00,283.200,283.200,280.610,281.290,254.330000,7.160933e+04,COMP,ETH


In [29]:
df_shuffled.iloc[[0, 2, 3]]

,ts,open,high,low,close,volume,volumeUSD,token,chain
57,2021-12-03 09:00:00,232.691,232.889,230.01,232.195,29521.347000,6.821438e+06,SOL,SOL
243,2021-12-11 03:00:00,3970.740,3997.990,3956.05,3992.210,4685.686108,1.864206e+07,ETH,ETH
266,2021-12-12 02:00:00,196.790,197.390,195.08,195.760,767.701000,1.506255e+05,COMP,ETH


lastly, we can set the DataFrame index from a column, or remove an index into a column

In [30]:
df_shuffled.set_index('ts')

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-03 09:00:00,232.691,232.889,230.010,232.195,29521.347000,6.821438e+06,SOL,SOL
2021-12-03 19:00:00,233.318,233.673,230.005,232.190,1491.289000,3.465333e+05,AAVE,ETH
2021-12-11 03:00:00,3970.740,3997.990,3956.050,3992.210,4685.686108,1.864206e+07,ETH,ETH
2021-12-12 02:00:00,196.790,197.390,195.080,195.760,767.701000,1.506255e+05,COMP,ETH
2021-12-11 23:00:00,4028.670,4069.530,4011.160,4025.340,7077.641660,2.855341e+07,ETH,ETH
...,...,...,...,...,...,...,...,...
2021-12-07 16:00:00,194.570,197.407,194.366,196.343,2986.129000,5.842146e+05,AAVE,ETH
2021-12-08 17:00:00,4407.730,4426.350,4370.840,4411.090,11455.200928,5.041474e+07,ETH,ETH
2021-12-11 17:00:00,48788.860,49306.830,48718.730,49011.400,610.683584,2.995077e+07,BTC,BTC


In [31]:
df_shuffled.set_index('ts').reset_index()

,ts,open,high,low,close,volume,volumeUSD,token,chain
0,2021-12-03 09:00:00,232.691,232.889,230.010,232.195,29521.347000,6.821438e+06,SOL,SOL
1,2021-12-03 19:00:00,233.318,233.673,230.005,232.190,1491.289000,3.465333e+05,AAVE,ETH
2,2021-12-11 03:00:00,3970.740,3997.990,3956.050,3992.210,4685.686108,1.864206e+07,ETH,ETH
3,2021-12-12 02:00:00,196.790,197.390,195.080,195.760,767.701000,1.506255e+05,COMP,ETH
4,2021-12-11 23:00:00,4028.670,4069.530,4011.160,4025.340,7077.641660,2.855341e+07,ETH,ETH
...,...,...,...,...,...,...,...,...,...
1455,2021-12-07 16:00:00,194.570,197.407,194.366,196.343,2986.129000,5.842146e+05,AAVE,ETH
1456,2021-12-08 17:00:00,4407.730,4426.350,4370.840,4411.090,11455.200928,5.041474e+07,ETH,ETH
1457,2021-12-11 17:00:00,48788.860,49306.830,48718.730,49011.400,610.683584,2.995077e+07,BTC,BTC
1458,2021-12-07 14:00:00,50973.170,51145.760,50873.240,50974.970,490.110868,2.499938e+07,BTC,BTC


## DataFrame Filtering

Filtering a data frame is very similar to filtering a series.  We can filter on any set of columns, the filtering is done via indices.  For example, if we wanted to just get the data points for tokens on the ethereum chain:

In [32]:
df['chain'] == 'ETH'

ts
2021-12-01 00:00:00    False
2021-12-01 01:00:00    False
2021-12-01 02:00:00    False
2021-12-01 03:00:00    False
2021-12-01 04:00:00    False
                       ...  
2021-12-12 23:00:00     True
2021-12-13 00:00:00     True
2021-12-13 01:00:00     True
2021-12-13 02:00:00     True
2021-12-13 03:00:00     True
Name: chain, Length: 1460, dtype: bool

In [33]:
df.loc[df['chain'] == 'ETH']

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 00:00:00,4656.62,4672.43,4624.16,4634.95,6013.006735,2.793321e+07,ETH,ETH
2021-12-01 01:00:00,4636.43,4736.90,4605.49,4729.10,13819.061610,6.487693e+07,ETH,ETH
2021-12-01 02:00:00,4729.10,4729.10,4684.49,4695.78,7491.465440,3.524161e+07,ETH,ETH
2021-12-01 03:00:00,4695.78,4754.97,4672.30,4754.09,10530.834423,4.963273e+07,ETH,ETH
2021-12-01 04:00:00,4754.09,4774.74,4722.02,4764.59,12471.624735,5.924627e+07,ETH,ETH
...,...,...,...,...,...,...,...,...
2021-12-12 23:00:00,199.71,200.00,197.97,199.27,618.775000,1.230245e+05,COMP,ETH
2021-12-13 00:00:00,199.27,200.05,198.30,198.62,296.920000,5.911383e+04,COMP,ETH
2021-12-13 01:00:00,198.52,200.81,195.49,196.24,3363.459000,6.637839e+05,COMP,ETH


In [34]:
df.loc[df['chain'] == 'ETH', 'close']

ts
2021-12-01 00:00:00    4634.95
2021-12-01 01:00:00    4729.10
2021-12-01 02:00:00    4695.78
2021-12-01 03:00:00    4754.09
2021-12-01 04:00:00    4764.59
                        ...   
2021-12-12 23:00:00     199.27
2021-12-13 00:00:00     198.62
2021-12-13 01:00:00     196.24
2021-12-13 02:00:00     191.86
2021-12-13 03:00:00     192.11
Name: close, Length: 876, dtype: float64

## Deleting from Dataframes

We can select for all the things we'd like, but we can also drop both rows and columns.  This also works by index, i.e.:

In [35]:
df.drop(pd.to_datetime('2021-12-01 00:00:00'))

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,BTC
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,BTC
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,BTC
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,BTC
2021-12-01 05:00:00,57404.01,57460.42,57016.00,57084.36,566.037996,3.238116e+07,BTC,BTC
...,...,...,...,...,...,...,...,...
2021-12-12 23:00:00,199.71,200.00,197.97,199.27,618.775000,1.230245e+05,COMP,ETH
2021-12-13 00:00:00,199.27,200.05,198.30,198.62,296.920000,5.911383e+04,COMP,ETH
2021-12-13 01:00:00,198.52,200.81,195.49,196.24,3363.459000,6.637839e+05,COMP,ETH


In [36]:
df.drop(columns='volumeUSD')

,open,high,low,close,volume,token,chain
ts,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,BTC,BTC
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,BTC,BTC
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,BTC,BTC
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,BTC,BTC
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,BTC,BTC
...,...,...,...,...,...,...,...
2021-12-12 23:00:00,199.71,200.00,197.97,199.27,618.775000,COMP,ETH
2021-12-13 00:00:00,199.27,200.05,198.30,198.62,296.920000,COMP,ETH
2021-12-13 01:00:00,198.52,200.81,195.49,196.24,3363.459000,COMP,ETH


In [37]:
df.drop(['close', 'open'], axis=1)

,high,low,volume,volumeUSD,token,chain
ts,,,,,,
2021-12-01 00:00:00,57451.05,56814.34,388.482022,2.218430e+07,BTC,BTC
2021-12-01 01:00:00,57726.45,56705.06,599.791578,3.437153e+07,BTC,BTC
2021-12-01 02:00:00,57620.00,56972.97,591.687200,3.387067e+07,BTC,BTC
2021-12-01 03:00:00,57396.87,56841.01,702.560364,4.007816e+07,BTC,BTC
2021-12-01 04:00:00,57456.82,57026.11,859.591535,4.920503e+07,BTC,BTC
...,...,...,...,...,...,...
2021-12-12 23:00:00,200.00,197.97,618.775000,1.230245e+05,COMP,ETH
2021-12-13 00:00:00,200.05,198.30,296.920000,5.911383e+04,COMP,ETH
2021-12-13 01:00:00,200.81,195.49,3363.459000,6.637839e+05,COMP,ETH


## Common Operations

Like with pandas Series, a DataFrame is simply a numpy array underneath the hood.

In [38]:
type(df.values)

numpy.ndarray

This means that the operations we saw for pandas Series can be applied to DataFrames as well, e.g. we can apply a scalar to every element in the DataFrame

In [39]:
df.head() * 10

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 00:00:00,573214.1,574510.5,568143.4,569879.7,3884.820220,2.218430e+08,BTCBTCBTCBTCBTCBTCBTCBTCBTCBTC,BTCBTCBTCBTCBTCBTCBTCBTCBTCBTC
2021-12-01 01:00:00,569983.5,577264.5,567050.6,576164.1,5997.915776,3.437153e+08,BTCBTCBTCBTCBTCBTCBTCBTCBTCBTC,BTCBTCBTCBTCBTCBTCBTCBTCBTCBTC
2021-12-01 02:00:00,576185.5,576200.0,569729.7,570308.3,5916.872000,3.387067e+08,BTCBTCBTCBTCBTCBTCBTCBTCBTCBTC,BTCBTCBTCBTCBTCBTCBTCBTCBTCBTC
2021-12-01 03:00:00,570297.9,573968.7,568410.1,573075.9,7025.603645,4.007816e+08,BTCBTCBTCBTCBTCBTCBTCBTCBTCBTC,BTCBTCBTCBTCBTCBTCBTCBTCBTCBTC
2021-12-01 04:00:00,573065.5,574568.2,570261.1,574040.1,8595.915349,4.920503e+08,BTCBTCBTCBTCBTCBTCBTCBTCBTCBTC,BTCBTCBTCBTCBTCBTCBTCBTCBTCBTC


However, the operation needs to be valid for ALL elements if we want to do this - e.g. while `*` is overridden for strings, `/` is not and will fail

In [40]:
df.head() / 10

TypeError: unsupported operand type(s) for /: 'str' and 'int'

Aggergation functions are by default done by column

In [41]:
df.mean()

/var/folders/ql/dg9s0kfx7sxg_byk_7xbqbgr0000gn/T/ipykernel_4412/3698961737.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.mean()


open         1.120936e+04
high         1.127694e+04
low          1.113114e+04
close        1.120278e+04
volume       1.698766e+04
volumeUSD    1.916371e+07
dtype: float64

However we can also make them aggregate by row:

In [42]:
df.mean(axis=1)

/var/folders/ql/dg9s0kfx7sxg_byk_7xbqbgr0000gn/T/ipykernel_4412/3676274908.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.mean(axis=1)


ts
2021-12-01 00:00:00    3.735544e+06
2021-12-01 01:00:00    5.766863e+06
2021-12-01 02:00:00    5.683417e+06
2021-12-01 03:00:00    6.717907e+06
2021-12-01 04:00:00    8.239181e+06
                           ...     
2021-12-12 23:00:00    2.074004e+04
2021-12-13 00:00:00    1.003450e+04
2021-12-13 01:00:00    1.113231e+05
2021-12-13 02:00:00    6.207089e+04
2021-12-13 03:00:00    1.478275e+04
Length: 1460, dtype: float64

## Mutating the Dataframe

Like with other functionality, mutating DataFrames is very similar to mutating Series.  For example, setting one column to a single value is easy:

In [43]:
df_mutations = df_base.set_index('ts')

In [44]:
df_mutations.head()

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,BTC
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,BTC
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,BTC
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,BTC
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,BTC


In [45]:
df_mutations['chain'] = 'NA'

In [46]:
df_mutations.head()

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,NA
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,NA
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,NA
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,NA
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,NA


We can also create a new column and add data by index:

In [47]:
updates = pd.Series({pd.to_datetime('2021-12-01 00:00:00'): 1})
updates

2021-12-01    1
dtype: int64

In [48]:
df_mutations['start_of_week'] = updates

In [49]:
df_mutations.head()

,open,high,low,close,volume,volumeUSD,token,chain,start_of_week
ts,,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,NA,1.0
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,NA,NaN
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,NA,NaN
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,NA,NaN
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,NA,NaN


We can also use the `.assign(...)` method to update columns, e.g.:

In [50]:
df_mutations.assign(
    chain=np.where(df_mutations.token.isin(['ETH', 'AAVE', 'COMP']), np.full(df_mutations.shape[0], 'ETH'), df_mutations.token),
    start_of_week=np.NaN
)

,open,high,low,close,volume,volumeUSD,token,chain,start_of_week
ts,,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,BTC,NaN
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,BTC,NaN
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,BTC,NaN
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,BTC,NaN
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,BTC,NaN
...,...,...,...,...,...,...,...,...,...
2021-12-12 23:00:00,199.71,200.00,197.97,199.27,618.775000,1.230245e+05,COMP,ETH,NaN
2021-12-13 00:00:00,199.27,200.05,198.30,198.62,296.920000,5.911383e+04,COMP,ETH,NaN
2021-12-13 01:00:00,198.52,200.81,195.49,196.24,3363.459000,6.637839e+05,COMP,ETH,NaN


**note**: using the index notation `[*]` will mutate the dataframe in place, however `.assign` will return a new data frame

We can also rename columns using a `{from:to}` syntax, e.g.:

In [51]:
df_mutations.rename(
    columns={
        'open':'OpeningPrice',
        'chain':'CryptoChain'
    }
)

,OpeningPrice,high,low,close,volume,volumeUSD,token,CryptoChain,start_of_week
ts,,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,NA,1.0
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,NA,NaN
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,NA,NaN
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,NA,NaN
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,NA,NaN
...,...,...,...,...,...,...,...,...,...
2021-12-12 23:00:00,199.71,200.00,197.97,199.27,618.775000,1.230245e+05,COMP,NA,NaN
2021-12-13 00:00:00,199.27,200.05,198.30,198.62,296.920000,5.911383e+04,COMP,NA,NaN
2021-12-13 01:00:00,198.52,200.81,195.49,196.24,3363.459000,6.637839e+05,COMP,NA,NaN


We can also use functions to rename, e.g.:

In [52]:
df_mutations.rename(columns=lambda x: x.upper())

,OPEN,HIGH,LOW,CLOSE,VOLUME,VOLUMEUSD,TOKEN,CHAIN,START_OF_WEEK
ts,,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,NA,1.0
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,NA,NaN
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,NA,NaN
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,NA,NaN
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,NA,NaN
...,...,...,...,...,...,...,...,...,...
2021-12-12 23:00:00,199.71,200.00,197.97,199.27,618.775000,1.230245e+05,COMP,NA,NaN
2021-12-13 00:00:00,199.27,200.05,198.30,198.62,296.920000,5.911383e+04,COMP,NA,NaN
2021-12-13 01:00:00,198.52,200.81,195.49,196.24,3363.459000,6.637839e+05,COMP,NA,NaN


The above commands will return a new DataFrame.  If we want to rename the input DataFrame, we can use the `inplace` option (which is available on most mutating functions), such as:

In [53]:
df_mutations.rename(
    columns={
        'open':'OpeningPrice',
        'chain':'CryptoChain'
    },
    inplace=True
)

df_mutations

,OpeningPrice,high,low,close,volume,volumeUSD,token,CryptoChain,start_of_week
ts,,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,NA,1.0
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,NA,NaN
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,NA,NaN
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,NA,NaN
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,NA,NaN
...,...,...,...,...,...,...,...,...,...
2021-12-12 23:00:00,199.71,200.00,197.97,199.27,618.775000,1.230245e+05,COMP,NA,NaN
2021-12-13 00:00:00,199.27,200.05,198.30,198.62,296.920000,5.911383e+04,COMP,NA,NaN
2021-12-13 01:00:00,198.52,200.81,195.49,196.24,3363.459000,6.637839e+05,COMP,NA,NaN


We can also add rows to the DataFrame by using `append`:

In [54]:
df_mutations.append(
    pd.Series({
        'high': 1,
        'low': 2,
        'token': 'FAKE'
    }, name=pd.to_datetime('2021-11-30 00:00:00'))
)

,OpeningPrice,high,low,close,volume,volumeUSD,token,CryptoChain,start_of_week
ts,,,,,,,,,
2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,NA,1.0
2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,NA,NaN
2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,NA,NaN
2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,NA,NaN
2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,NA,NaN
...,...,...,...,...,...,...,...,...,...
2021-12-13 00:00:00,199.27,200.05,198.30,198.62,296.920000,5.911383e+04,COMP,NA,NaN
2021-12-13 01:00:00,198.52,200.81,195.49,196.24,3363.459000,6.637839e+05,COMP,NA,NaN
2021-12-13 02:00:00,196.31,196.32,191.80,191.86,1901.019000,3.697480e+05,COMP,NA,NaN


## Sorting DataFrames

One thing that we didn't need to really do with Series is sorting.  For DataFrames, we will often need to sort by column(s) or by the index.  We can use `sort_values` and `sort_index` to do this

In [55]:
df.sort_values('open')

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-11 02:00:00,165.28,171.32,164.68,168.19,80534.167000,1.351489e+07,SOL,SOL
2021-12-11 01:00:00,166.98,170.41,162.92,167.91,6946.601000,1.159482e+06,AAVE,ETH
2021-12-11 01:00:00,167.63,169.86,161.51,165.32,276907.405000,4.597063e+07,SOL,SOL
2021-12-11 02:00:00,167.75,171.36,167.46,169.45,1740.741000,2.957224e+05,AAVE,ETH
2021-12-11 04:00:00,168.00,172.00,167.68,170.27,57054.990000,9.721131e+06,SOL,SOL
...,...,...,...,...,...,...,...,...
2021-12-01 15:00:00,57706.57,58783.16,57704.98,58610.19,908.858596,5.303393e+07,BTC,BTC
2021-12-01 19:00:00,58037.52,58148.15,57423.59,57498.26,851.152400,4.917105e+07,BTC,BTC
2021-12-01 18:00:00,58485.88,58631.40,58007.24,58037.51,633.239868,3.697725e+07,BTC,BTC


In [56]:
df.sort_values('open', ascending=False)

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 17:00:00,58664.40,59118.84,58445.53,58485.88,728.515015,4.285684e+07,BTC,BTC
2021-12-01 16:00:00,58610.19,58900.00,58349.19,58664.40,684.590976,4.012464e+07,BTC,BTC
2021-12-01 18:00:00,58485.88,58631.40,58007.24,58037.51,633.239868,3.697725e+07,BTC,BTC
2021-12-01 19:00:00,58037.52,58148.15,57423.59,57498.26,851.152400,4.917105e+07,BTC,BTC
2021-12-01 15:00:00,57706.57,58783.16,57704.98,58610.19,908.858596,5.303393e+07,BTC,BTC
...,...,...,...,...,...,...,...,...
2021-12-11 04:00:00,168.00,172.00,167.68,170.27,57054.990000,9.721131e+06,SOL,SOL
2021-12-11 02:00:00,167.75,171.36,167.46,169.45,1740.741000,2.957224e+05,AAVE,ETH
2021-12-11 01:00:00,167.63,169.86,161.51,165.32,276907.405000,4.597063e+07,SOL,SOL


In [57]:
df.sort_values(['volumeUSD', 'open'])

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-02 08:00:00,270.71,272.13,270.58,270.89,67.178000,1.821233e+04,COMP,ETH
2021-12-12 08:00:00,195.65,195.98,194.83,195.27,134.849000,2.635749e+04,COMP,ETH
2021-12-01 10:00:00,283.07,284.40,282.27,284.10,123.910000,3.510889e+04,COMP,ETH
2021-12-12 08:00:00,179.22,179.62,178.58,179.41,211.370000,3.787862e+04,AAVE,ETH
2021-12-02 11:00:00,270.56,271.60,269.00,269.85,151.092000,4.080482e+04,COMP,ETH
...,...,...,...,...,...,...,...,...
2021-12-03 21:00:00,53842.35,54060.00,51640.00,53494.99,3584.238130,1.905032e+08,BTC,BTC
2021-12-03 17:00:00,4469.68,4469.68,4320.00,4344.51,44978.935447,1.973315e+08,ETH,ETH
2021-12-03 21:00:00,4229.09,4229.09,4035.00,4212.34,51723.840085,2.147795e+08,ETH,ETH


We can also sort by the index

In [58]:
df.sort_index()

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-01 00:00:00,57321.410,57451.050,56814.340,56987.970,388.482022,2.218430e+07,BTC,BTC
2021-12-01 00:00:00,4656.620,4672.430,4624.160,4634.950,6013.006735,2.793321e+07,ETH,ETH
2021-12-01 00:00:00,257.102,260.775,255.345,257.078,2730.299000,7.039183e+05,AAVE,ETH
2021-12-01 00:00:00,210.312,210.590,208.432,208.676,70031.618000,1.465851e+07,SOL,SOL
2021-12-01 00:00:00,280.590,281.400,278.300,278.700,207.849000,5.822157e+04,COMP,ETH
...,...,...,...,...,...,...,...,...
2021-12-13 03:00:00,173.700,175.250,173.510,174.170,647.711000,1.129002e+05,AAVE,ETH
2021-12-13 03:00:00,3970.920,3999.900,3954.270,3979.890,7376.954839,2.936103e+07,ETH,ETH
2021-12-13 03:00:00,48625.720,48912.180,48464.660,48756.750,448.845398,2.186516e+07,BTC,BTC


## Grouping DataFrames

one _very common_ action we will do during data manipulation is grouping then aggregating.  Pandas DataFrame has the method `groupby`, which allows us to group by any column in our DataFrame.

`groupby` returns a `DataFrameGroupBy` object, which we can apply a function to each group, or directly aggregate

In [59]:
df.groupby('chain')

In [60]:
df.groupby('chain').groups

{'BTC': [2021-12-01 00:00:00, 2021-12-01 01:00:00, 2021-12-01 02:00:00, 2021-12-01 03:00:00, 2021-12-01 04:00:00, 2021-12-01 05:00:00, 2021-12-01 06:00:00, 2021-12-01 07:00:00, 2021-12-01 08:00:00, 2021-12-01 09:00:00, 2021-12-01 10:00:00, 2021-12-01 11:00:00, 2021-12-01 12:00:00, 2021-12-01 13:00:00, 2021-12-01 14:00:00, 2021-12-01 15:00:00, 2021-12-01 16:00:00, 2021-12-01 17:00:00, 2021-12-01 18:00:00, 2021-12-01 19:00:00, 2021-12-01 20:00:00, 2021-12-01 21:00:00, 2021-12-01 22:00:00, 2021-12-01 23:00:00, 2021-12-02 00:00:00, 2021-12-02 01:00:00, 2021-12-02 02:00:00, 2021-12-02 03:00:00, 2021-12-02 04:00:00, 2021-12-02 05:00:00, 2021-12-02 06:00:00, 2021-12-02 07:00:00, 2021-12-02 08:00:00, 2021-12-02 09:00:00, 2021-12-02 10:00:00, 2021-12-02 11:00:00, 2021-12-02 12:00:00, 2021-12-02 13:00:00, 2021-12-02 14:00:00, 2021-12-02 15:00:00, 2021-12-02 16:00:00, 2021-12-02 17:00:00, 2021-12-02 18:00:00, 2021-12-02 19:00:00, 2021-12-02 20:00:00, 2021-12-02 21:00:00, 2021-12-02 22:00:00, 2021

In [61]:
len(df.groupby('chain'))

3

In [62]:
df.groupby('chain').size()

chain
BTC    292
ETH    876
SOL    292
dtype: int64

after grouping, we can operate on the whole DataFrame or on any column

In [63]:
df.groupby('chain')['volumeUSD'].sum().to_frame()

,volumeUSD
chain,
BTC,1.091870e+10
ETH,1.300667e+10
SOL,4.053649e+09


we can also groupby multiple columns.  The row indices now are a multi-index, however we will not go into this

In [64]:
df.groupby(['chain', 'token'])['volumeUSD'].sum().to_frame()

volumeUSD
chain token              
BTC   BTC    1.091870e+10
ETH   AAVE   1.430826e+08
      COMP   7.253554e+07
      ETH    1.279105e+10
SOL   SOL    4.053649e+09

We can actually aggregate without setting a compound index by adding `as_index=False`

In [65]:
df.groupby(['chain', 'token'], as_index=False)['volumeUSD'].sum()

,chain,token,volumeUSD
0,BTC,BTC,1.091870e+10
1,ETH,AAVE,1.430826e+08
2,ETH,COMP,7.253554e+07
3,ETH,ETH,1.279105e+10
4,SOL,SOL,4.053649e+09


We can now operate on the groups.  For example, if we wanted to sum all columns:

In [66]:
df.groupby('chain').aggregate(np.sum)

,open,high,low,close,volume,volumeUSD
chain,,,,,,
BTC,1.493860e+07,1.502706e+07,1.483654e+07,1.492990e+07,2.153074e+05,1.091870e+10
ETH,1.369844e+06,1.379454e+06,1.358360e+06,1.368992e+06,4.093601e+06,1.300667e+10
SOL,5.721107e+04,5.781438e+04,5.656171e+04,5.716828e+04,2.049307e+07,4.053649e+09


or describe all columns

In [67]:
df.groupby('chain').describe()

open                                                                \
       count          mean          std       min          25%         50%   
chain                                                                        
BTC    292.0  51159.604247  3408.256653  46588.66  48684.55250  49612.0350   
ETH    876.0   1563.749171  1915.277243    166.98    195.04875    232.4190   
SOL    292.0    195.928322    20.239565    165.28    180.54250    192.5405   

                               high                ...        volume  \
              75%        max  count          mean  ...           75%   
chain                                              ...                 
BTC    53446.8775  58664.400  292.0  51462.540171  ...    919.558897   
ETH     4064.7750   4772.600  876.0   1574.719150  ...   5781.047960   
SOL      209.1150    242.099  292.0    197.994445  ...  84399.865750   

                     volumeUSD                                            \
                 max     count          mean           std           min   
chain                                                                      
BTC      8420.715164     292.0  3.739282e+07  3.127329e+07  8.737496e+06   
ETH     97097.417492     876.0  1.484779e+07  2.908290e+07  1.821233e+04   
SOL    534212.095000     292.0  1.388236e+07  9.829576e+06  1.781884e+06   

                                                               
                25%           50%           75%           max  
chain                                                          
BTC    2.017780e+07  3.208078e+07  4.614977e+07  3.988035e+08  
ETH    1.969494e+05  4.621537e+05  2.194393e+07  3.702769e+08  
SOL    7.776958e+06  1.200529e+07  1.680370e+07  9.940887e+07  

[3 rows x 48 columns]

We can also do multiple aggregations

In [68]:
df.groupby('chain')['open'].agg([np.size, np.mean, np.std, np.min, np.max])

,size,mean,std,amin,amax
chain,,,,,
BTC,292,51159.604247,3408.256653,46588.66,58664.400
ETH,876,1563.749171,1915.277243,166.98,4772.600
SOL,292,195.928322,20.239565,165.28,242.099


we can actually use _any_ arbitrary functions - for example, we can use lambdas

In [69]:
df.groupby('chain')['open'].agg(
    range=lambda x: x.max() - x.min()
)

,range
chain,
BTC,12075.740
ETH,4605.620
SOL,76.819


## Joining Dataframes

One of the primary things we need to do before starting to clean data is to make sure that we can get all of our data into one place.  This is usually called either a fat talbe or a long table, depending on how we are doing the joining.  We'll look at a few different ways to join pandas DataFrames below.

We will be using `dfs`, which is a list of DataFrames that we created up above

### `pd.concat`

To join the dataframes lengthwise, we can use `pd.concat`.  This will append the dataframes together, and join the rows by using the column names as an index.  If any dataframe doesn't have a column name another one has, it will appear in the full DataFrame but will have NA for the missing DataFrames

In [70]:
pd.concat(dfs)

,ts,open,high,low,close,volume,volumeUSD,token,chain
0,2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,BTC
1,2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,BTC
2,2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,BTC
3,2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,BTC
4,2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,BTC
...,...,...,...,...,...,...,...,...,...
287,2021-12-12 23:00:00,199.71,200.00,197.97,199.27,618.775000,1.230245e+05,COMP,ETH
288,2021-12-13 00:00:00,199.27,200.05,198.30,198.62,296.920000,5.911383e+04,COMP,ETH
289,2021-12-13 01:00:00,198.52,200.81,195.49,196.24,3363.459000,6.637839e+05,COMP,ETH
290,2021-12-13 02:00:00,196.31,196.32,191.80,191.86,1901.019000,3.697480e+05,COMP,ETH


if you wanted to make sure you know where the original data is from, we can add keys, which creates a multi index:

In [71]:
res = pd.concat(dfs, keys=tokens)
res

ts      open      high       low     close  \
BTC  0   2021-12-01 00:00:00  57321.41  57451.05  56814.34  56987.97   
     1   2021-12-01 01:00:00  56998.35  57726.45  56705.06  57616.41   
     2   2021-12-01 02:00:00  57618.55  57620.00  56972.97  57030.83   
     3   2021-12-01 03:00:00  57029.79  57396.87  56841.01  57307.59   
     4   2021-12-01 04:00:00  57306.55  57456.82  57026.11  57404.01   
...                      ...       ...       ...       ...       ...   
COMP 287 2021-12-12 23:00:00    199.71    200.00    197.97    199.27   
     288 2021-12-13 00:00:00    199.27    200.05    198.30    198.62   
     289 2021-12-13 01:00:00    198.52    200.81    195.49    196.24   
     290 2021-12-13 02:00:00    196.31    196.32    191.80    191.86   
     291 2021-12-13 03:00:00    191.69    193.47    190.95    192.11   

               volume     volumeUSD token chain  
BTC  0     388.482022  2.218430e+07   BTC   BTC  
     1     599.791578  3.437153e+07   BTC   BTC  
     2     591.687200  3.387067e+07   BTC   BTC  
     3     702.560364  4.007816e+07   BTC   BTC  
     4     859.591535  4.920503e+07   BTC   BTC  
...               ...           ...   ...   ...  
COMP 287   618.775000  1.230245e+05  COMP   ETH  
     288   296.920000  5.911383e+04  COMP   ETH  
     289  3363.459000  6.637839e+05  COMP   ETH  
     290  1901.019000  3.697480e+05  COMP   ETH  
     291   454.858000  8.747343e+04  COMP   ETH  

[1460 rows x 9 columns]

this allows us to select the data from the source tables, e.g.:

In [72]:
res.loc['COMP']

,ts,open,high,low,close,volume,volumeUSD,token,chain
0,2021-12-01 00:00:00,280.59,281.40,278.30,278.70,207.849,58221.57184,COMP,ETH
1,2021-12-01 01:00:00,278.65,283.80,276.36,283.44,817.668,229274.61550,COMP,ETH
2,2021-12-01 02:00:00,283.20,283.20,280.61,281.29,254.330,71609.32568,COMP,ETH
3,2021-12-01 03:00:00,281.25,283.22,279.90,283.09,393.771,110890.74168,COMP,ETH
4,2021-12-01 04:00:00,283.10,284.16,282.60,283.73,489.120,138532.76788,COMP,ETH
...,...,...,...,...,...,...,...,...,...
287,2021-12-12 23:00:00,199.71,200.00,197.97,199.27,618.775,123024.49025,COMP,ETH
288,2021-12-13 00:00:00,199.27,200.05,198.30,198.62,296.920,59113.82992,COMP,ETH
289,2021-12-13 01:00:00,198.52,200.81,195.49,196.24,3363.459,663783.90199,COMP,ETH
290,2021-12-13 02:00:00,196.31,196.32,191.80,191.86,1901.019,369748.01226,COMP,ETH


As we saw above, we can also use `.append(*)` on DataFrames as well as Series

In [73]:
dfs[0].append(dfs[1])

,ts,open,high,low,close,volume,volumeUSD,token,chain
0,2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,BTC
1,2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,BTC
2,2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,BTC
3,2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,BTC
4,2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,BTC
...,...,...,...,...,...,...,...,...,...
287,2021-12-12 23:00:00,4124.78,4158.00,4116.49,4156.67,3011.092519,1.245128e+07,ETH,ETH
288,2021-12-13 00:00:00,4156.84,4172.00,4127.40,4132.92,5090.836084,2.113033e+07,ETH,ETH
289,2021-12-13 01:00:00,4132.91,4146.10,4056.82,4069.44,7688.171067,3.153781e+07,ETH,ETH
290,2021-12-13 02:00:00,4070.58,4070.58,3961.85,3970.92,18723.615566,7.507152e+07,ETH,ETH


Lastly, remember the importance of indices.  In the operation above (both `.concat` and `.append`) we joined the DataFrames while keeping the indices of the original tables.  This means that we have repeated indices:

In [74]:
dfs[0].append(dfs[1]).sort_index()

,ts,open,high,low,close,volume,volumeUSD,token,chain
0,2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,BTC
0,2021-12-01 00:00:00,4656.62,4672.43,4624.16,4634.95,6013.006735,2.793321e+07,ETH,ETH
1,2021-12-01 01:00:00,4636.43,4736.90,4605.49,4729.10,13819.061610,6.487693e+07,ETH,ETH
1,2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,BTC
2,2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,BTC
...,...,...,...,...,...,...,...,...,...
289,2021-12-13 01:00:00,50089.64,50218.42,49570.70,49652.89,419.303095,2.091323e+07,BTC,BTC
290,2021-12-13 02:00:00,4070.58,4070.58,3961.85,3970.92,18723.615566,7.507152e+07,ETH,ETH
290,2021-12-13 02:00:00,49663.75,49664.16,48595.01,48624.41,929.600989,4.568805e+07,BTC,BTC
291,2021-12-13 03:00:00,48625.72,48912.18,48464.66,48758.38,448.678884,2.185704e+07,BTC,BTC


This sometimes isn't ideal, esp. if we want to join against these indices later.  Instead, we can create a new index on the joined table using the `ignore_index` parameter, which allows us to have a sequential, non-repeated index:

In [75]:
dfs[0].append(dfs[1], ignore_index=True).sort_index()

,ts,open,high,low,close,volume,volumeUSD,token,chain
0,2021-12-01 00:00:00,57321.41,57451.05,56814.34,56987.97,388.482022,2.218430e+07,BTC,BTC
1,2021-12-01 01:00:00,56998.35,57726.45,56705.06,57616.41,599.791578,3.437153e+07,BTC,BTC
2,2021-12-01 02:00:00,57618.55,57620.00,56972.97,57030.83,591.687200,3.387067e+07,BTC,BTC
3,2021-12-01 03:00:00,57029.79,57396.87,56841.01,57307.59,702.560364,4.007816e+07,BTC,BTC
4,2021-12-01 04:00:00,57306.55,57456.82,57026.11,57404.01,859.591535,4.920503e+07,BTC,BTC
...,...,...,...,...,...,...,...,...,...
579,2021-12-12 23:00:00,4124.78,4158.00,4116.49,4156.67,3011.092519,1.245128e+07,ETH,ETH
580,2021-12-13 00:00:00,4156.84,4172.00,4127.40,4132.92,5090.836084,2.113033e+07,ETH,ETH
581,2021-12-13 01:00:00,4132.91,4146.10,4056.82,4069.44,7688.171067,3.153781e+07,ETH,ETH
582,2021-12-13 02:00:00,4070.58,4070.58,3961.85,3970.92,18723.615566,7.507152e+07,ETH,ETH


### `pd.DataFrame.join`

`df.join` is a nice and easy method that allows us to join two dataframes by their index

In [76]:
dfs[0].set_index('ts')['close'].rename(f'close_{tokens[0]}').to_frame().join(
    dfs[1].set_index('ts')['close'].rename(f'close_{tokens[1]}').to_frame()
)

,close_BTC,close_ETH
ts,,
2021-12-01 00:00:00,56987.97,4634.95
2021-12-01 01:00:00,57616.41,4729.10
2021-12-01 02:00:00,57030.83,4695.78
2021-12-01 03:00:00,57307.59,4754.09
2021-12-01 04:00:00,57404.01,4764.59
...,...,...
2021-12-12 23:00:00,50317.33,4156.67
2021-12-13 00:00:00,50089.64,4132.92
2021-12-13 01:00:00,49652.89,4069.44


we can get a little more advanced by having a **left** unkeyed DataFrame joining against a **right** keyed DataFrame, e.g.:

In [77]:
dfs[0][['ts', 'close']].join(
    dfs[1].set_index('ts')['close'].rename(f'close_{tokens[1]}').to_frame(),
    on='ts'
)

,ts,close,close_ETH
0,2021-12-01 00:00:00,56987.97,4634.95
1,2021-12-01 01:00:00,57616.41,4729.10
2,2021-12-01 02:00:00,57030.83,4695.78
3,2021-12-01 03:00:00,57307.59,4754.09
4,2021-12-01 04:00:00,57404.01,4764.59
...,...,...,...
287,2021-12-12 23:00:00,50317.33,4156.67
288,2021-12-13 00:00:00,50089.64,4132.92
289,2021-12-13 01:00:00,49652.89,4069.44
290,2021-12-13 02:00:00,48624.41,3970.92


### `pd.merge`

`pd.merge` is Pandas way of doing sql-like joins (e.g. left join, inner join, outer join etc).  There are a few quirks we'll see though.

In [78]:
pd.merge(
    dfs[0][['ts', 'close']].rename(columns={'close': f'close_{tokens[0]}'}),
    dfs[1][['ts', 'close']].rename(columns={'close': f'close_{tokens[1]}'}),
    on='ts',
    how='inner'
)

,ts,close_BTC,close_ETH
0,2021-12-01 00:00:00,56987.97,4634.95
1,2021-12-01 01:00:00,57616.41,4729.10
2,2021-12-01 02:00:00,57030.83,4695.78
3,2021-12-01 03:00:00,57307.59,4754.09
4,2021-12-01 04:00:00,57404.01,4764.59
...,...,...,...
287,2021-12-12 23:00:00,50317.33,4156.67
288,2021-12-13 00:00:00,50089.64,4132.92
289,2021-12-13 01:00:00,49652.89,4069.44
290,2021-12-13 02:00:00,48624.41,3970.92


we can use other conditions for `how`, e.g. 'left', 'right', 'outer', and 'cross'

if left and right DataFrames have columns with the same name, pandas will automatically resolve the delta by adding `_x` and `_y` suffixes to the conflicted columns

In [79]:
pd.merge(
    dfs[0][['ts', 'close']],
    dfs[1][['ts', 'close']],
    on='ts',
    how='inner'
)

,ts,close_x,close_y
0,2021-12-01 00:00:00,56987.97,4634.95
1,2021-12-01 01:00:00,57616.41,4729.10
2,2021-12-01 02:00:00,57030.83,4695.78
3,2021-12-01 03:00:00,57307.59,4754.09
4,2021-12-01 04:00:00,57404.01,4764.59
...,...,...,...
287,2021-12-12 23:00:00,50317.33,4156.67
288,2021-12-13 00:00:00,50089.64,4132.92
289,2021-12-13 01:00:00,49652.89,4069.44
290,2021-12-13 02:00:00,48624.41,3970.92


however, we can also define our own suffixes, e.g.

In [ ]:
pd.merge(
    dfs[0][['ts', 'close']],
    dfs[1][['ts', 'close']],
    on='ts',
    how='inner',
    suffixes=[f'_{tokens[0]}', f'_{tokens[1]}']
)